## Import

In [ ]:
import momepy
import numpy as np
import osmnx as ox
import pandas as pd
import networkx as nx
import geopandas as gpd

from tqdm import tqdm
from typing import Optional

## Functions

### Загрузка GDF из OSM

In [ ]:
def download_osm(id, tags=False):
    geocode_to_gdf = ox.geocode_to_gdf(id, by_osmid=True)
    polygon_boundary = geocode_to_gdf.unary_union
    if tags:
        gdf = ox.features_from_polygon(polygon_boundary, tags=tags)
    else:
        gdf = ox.features_from_polygon(polygon_boundary)
    return gdf

# tags = {'highway': ['motorway', 'trunk', 'primary', 
#                     'secondary','tertiary', 'unclassified', 
#                     'residential', 'motorway_link', 'trunk_link', 
#                     'primary_link', 'secondary_link', 'tertiary_link']}
# gdf_osm = download_osm(['R176095'], tags)

### Плотность УДС $км/км^2$

In [ ]:
def density_roads(gdf_polygon: gpd.GeoDataFrame, gdf_line: gpd.GeoDataFrame) -> float:
    area = gdf_polygon.to_crs(epsg=3857).unary_union.area / 1000000
    length = gdf_line.geometry.length.sum()
    print(f'Плотность: {length / area:.3f} км/км^2')

    return round(length / area, 3)

### Протяженность дорог каждого типа

In [ ]:
def calculate_length_sum_by_status(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    gdf = gdf.to_crs(epsg=3857)
    gdf['REG_STATUS'] = gdf['REG_STATUS'].fillna(3)
    length_sum_by_status = gdf.groupby('REG_STATUS').geometry.apply(lambda x: x.length.sum() / 1000)
    print(length_sum_by_status.reset_index())
    
    return length_sum_by_status.reset_index()

## Work place

### Вроде, основная область

In [ ]:
# edges_spb.select_dtypes(include=[np.datetime64])[edges_spb.select_dtypes(include=[np.datetime64]).notna().any(axis=1)]

In [ ]:
# with open("data/test.geojson","w", encoding='utf-8') as f:
#     f.write(edges_spb.drop(columns=['check_date', 'check_da_1']).to_crs(epsg=4326).to_json(na="drop", ensure_ascii=False))

# test_geo = gpd.read_file('data/test.geojson')
# test_geo.to_crs(epsg=3857, inplace=True)
# test_geo['length'] = test_geo.geometry.length

In [ ]:
# edges_spb = gpd.read_file('data/geojsons/edges_spb.geojson')                                # Наш файл дорог
edges_spb = gpd.read_parquet('data/parquets/УДС без будущих дорог в crs 32636.parquet')     # Наш файл дорог
gdf_polygon = gpd.read_file('data/geojsons/Границы_только_МР_Границы_ЛО_Без_воды.geojson')  # Наташин полигон без воды
gdf_3000_points = gpd.read_file("data/geojsons/СНП ЛО.geojson")                             # Куча центров

In [ ]:
# Предпроцессинг типа
gdf_3000_points.to_crs(epsg=32636, inplace=True)
gdf_polygon.to_crs(epsg=32636, inplace=True)
edges_spb.to_crs(epsg=32636, inplace=True)
edges_spb['length'] = edges_spb.geometry.length

# Административные центры
cities = ["приозерск", "кировск", "кингисепп", "луга", 
          "лодейное поле", "гатчина", "тихвин", "тосно", 
          "выборг", "бокситогорск", "всеволожск", "волосово", 
          "волхов", "сосновый бор", "сланцы", "подпорожье", "кириши"]
cities_lower = [city.lower() for city in cities]

# Выделение административных центров
gdf_17_points = gdf_3000_points[
    (gdf_3000_points['name'].str.lower().isin(cities_lower)) &
    (gdf_3000_points['rural settlement'].str.contains('административный центр'))
].sort_values('name')


# 'how' определяет тип пространственного объединения: 'inner' означает, что в результат попадут только те объекты, которые пересекаются.
# 'predicate' определяет тип пространственного отношения: 'intersects', 'within', 'contains' и т.д.
roads_within_polygon = gpd.sjoin(edges_spb, gdf_polygon, how='inner', predicate='intersects')

In [ ]:
density = density_roads(gdf_polygon, roads_within_polygon)
length_roads = calculate_length_sum_by_status(roads_within_polygon)

In [ ]:
def transferring_points_on_line(
        gdf_points: gpd.GeoDataFrame, 
        gdf_lines:  gpd.GeoDataFrame,
        column_1:   Optional[str],
        column_2:   Optional[str]
        ) -> gpd.GeoDataFrame:
    
    gdf_points_copy = gdf_points.copy().reset_index(drop=True)
    gdf_lines_copy  = gdf_lines.copy().reset_index(drop=True)
    nearest_roads = gpd.sjoin_nearest(gdf_points_copy, gdf_lines_copy, how='left')
    if f'{column_1}_right' in nearest_roads.columns.values:
        gdf_lines_copy.loc[nearest_roads.index_right, column_2] = nearest_roads[f'{column_1}_left'].values
        return gdf_lines_copy
    else:
        gdf_lines_copy.loc[nearest_roads.index_right, column_2] = nearest_roads[column_1].values
        return gdf_lines_copy


gdf_lines = transferring_points_on_line(gdf_17_points, roads_within_polygon.drop(columns=['index_right']), column_1='name', column_2='admin_centers')

In [ ]:
gdf_lines_copy = gdf_lines.copy()
reg_status = gdf_lines_copy[['name', 'REG_STATUS', 'geometry', 'admin_centers']][gdf_lines_copy.REG_STATUS == 1].copy().reset_index()
admin_centers = gdf_lines_copy[['name', 'REG_STATUS', 'geometry', 'admin_centers']][(gdf_lines_copy.admin_centers.notna())].copy()

In [ ]:
buffer_for_city = {
    'Луга':             5000,
    'Волосово':         45000,
    'Сосновый бор':     55000,
    'Приозерск':        3000,
    'Гатчина':          10000,
    'Кировск':          2000,
    'Всеволожск':       25000,
    'Тосно':            1000,
    'Кириши':           100000,
    'Волхов':           20000,
    'Лодейное Поле':    5000,
    'Тихвин':           9000,
    'Бокситогорск':     30000,
    'Подпорожье':       80000,
    'Кингисепп':        5000,
    'Выборг':           8000,
    'Сланцы':           100000
}

# Инициализируйте словарь для хранения сопоставления id дороги и города
road_to_city_mapping = {}

for index, city in admin_centers.iterrows():
    # Создаем буфер вокруг каждого города
    buffer_meter = buffer_for_city[city['admin_centers']]
    buffer = city['geometry'].buffer(buffer_meter)
    roads_in_buffer = reg_status.loc[reg_status.geometry.intersects(buffer)]
    
    for road_id in roads_in_buffer['index']:
        # Если id дороги уже есть в словаре - добавить город к списку городов для этой id
        if road_id in road_to_city_mapping:
            road_to_city_mapping[road_id].append(city['admin_centers'])
        else:  # Если id дороги еще нет в словаре, создайте новый список городов с этим городом
            road_to_city_mapping[road_id] = [city['admin_centers']]
gdf_lines_copy['nearest_city'] = gdf_lines_copy.index.map(road_to_city_mapping)

In [ ]:
graph = momepy.gdf_to_nx(gdf_lines_copy.to_crs(epsg=4326))
    for node in graph.nodes:
        graph.nodes()[node]['x'] = node[0]
        graph.nodes()[node]['y'] = node[1]

# Для каждого ребра в графе
for u, v, data in graph.edges(data=True):
    if graph.nodes[u]:
        if graph.nodes[u].get('REG_STATUS') != 1:
            graph.nodes[u]['REG_STATUS'] = data['REG_STATUS']
        if isinstance(data['admin_centers'], str):
            graph.nodes[u]['admin_centers'] = data['admin_centers']
        if isinstance(data['nearest_city'], list) and data['REG_STATUS'] == 1:
            if 'nearest_city' not in graph.nodes[u]:
                graph.nodes[u]['nearest_city'] = data['nearest_city']
                

    if graph.nodes[v]:
        if graph.nodes[v].get('REG_STATUS') != 1:
            graph.nodes[v]['REG_STATUS'] = data['REG_STATUS']
        if isinstance(data['admin_centers'], str):
            graph.nodes[v]['admin_centers'] = data['admin_centers']
        if isinstance(data['nearest_city'], list) and data['REG_STATUS'] == 1:
            if 'nearest_city' not in graph.nodes[v]:
                graph.nodes[v]['nearest_city'] = data['nearest_city']

In [ ]:
def search_min_distance(graph):
    source_nodes = []
    target_nodes = []

    # Формируем список исходных и целевых узлов
    for node, data in graph.nodes(data=True):
        if isinstance(data.get('admin_centers'), str):  # Исходные узлы
            source_nodes.append((node, data['admin_centers']))
        if data.get('REG_STATUS') == 1 and isinstance(data.get('nearest_city'), list):  # Целевые узлы
            target_nodes.append((node, data['nearest_city']))

    min_distances = {} # Словарь для хранения минимальных расстояний

    # Общее количество итераций для отображения прогресса
    total_iterations = len(source_nodes) * len(target_nodes)

    progress_bar = tqdm(total=total_iterations)  # Инициализация индикатора прогресса

    # Поиск кратчайшего пути и запоминание минимального расстояния
    for source_node, name in source_nodes:
        for target_node, name_right in target_nodes:
            if graph.nodes[target_node].get('nearest_city') and name in graph.nodes[target_node]['nearest_city']:
                try: 
                    length, path = nx.single_source_dijkstra(graph, source=source_node, target=target_node, weight='length')
                    if name not in min_distances or (length/1000) < min_distances[name][0]:
                        min_distances[name] = (length/1000, path)
                except nx.NetworkXNoPath:
                    print(f"Путь до {name_right} не найден.")
            progress_bar.update()  # Обновление индикатора прогресса

    progress_bar.close()  # Закрытие индикатора прогресса

    # Преобразование словаря в DataFrame для более удобного отображения
    df = pd.DataFrame(min_distances.values(), index=min_distances.keys())
    df.columns = ['min_distance', 'path']
    return df

result_path_length = search_min_distance(graph)

In [ ]:
result_path_length.sort_values('min_distance')

In [ ]:
import folium
mymap = folium.Map()

edges = [(u,v,d) for u, v, d in graph.edges(data=True) if d['REG_STATUS'] == 1]

# Отображение на карте
for edge in edges:
    coordinates = [[graph.nodes[edge[0]]['y'], graph.nodes[edge[0]]['x']], 
                   [graph.nodes[edge[1]]['y'], graph.nodes[edge[1]]['x']]]
    folium.PolyLine(coordinates, color="blue", weight=2.5, opacity=1, tooltip=f"{graph.nodes[edge[0]]['REG_STATUS']}").add_to(mymap)


for index_city in range(len(result_path_length['path'])):
    for i in range(len(result_path_length['path'][index_city]) - 1):
        edge = (result_path_length['path'][index_city][i], result_path_length['path'][index_city][i+1])
        if i == 0:
            coordinates = [[graph.nodes[edge[0]]['y'], graph.nodes[edge[0]]['x']], 
                        [graph.nodes[edge[1]]['y'], graph.nodes[edge[1]]['x']]]
            folium.PolyLine(coordinates, color="yellow", weight=2.5, opacity=1).add_to(mymap)
        else:
            coordinates = [[graph.nodes[edge[0]]['y'], graph.nodes[edge[0]]['x']], 
                    [graph.nodes[edge[1]]['y'], graph.nodes[edge[1]]['x']]]
            folium.PolyLine(coordinates, color="red", weight=2.5, opacity=1).add_to(mymap)

mymap

In [ ]:
saint_petersburg_point = ox.geocode_to_gdf('N27490597', by_osmid=True)
saint_petersburg_point.to_crs(epsg=32636, inplace=True)
edges_spb.to_crs(epsg=32636, inplace=True)
edges_spb['length'] = edges_spb.geometry.length

In [ ]:
gdf_18_points = pd.concat([gdf_17_points.to_crs(epsg=32636), saint_petersburg_point[['name', 'geometry']]], ignore_index=True)
# gdf_lines = transferring_points_on_line(gdf_18_points, roads_within_polygon.drop(columns=['index_right', 'name']), 'name')
gdf_lines_spb = transferring_points_on_line(gdf_18_points, edges_spb, column_1='name', column_2='admin_centers')

In [ ]:
graph = momepy.gdf_to_nx(gdf_lines_spb.to_crs(epsg=4326))
for node in graph.nodes:
    graph.nodes()[node]['x'] = node[0]
    graph.nodes()[node]['y'] = node[1]

# Для каждого ребра в графе
for u, v, data in graph.edges(data=True):
    if graph.nodes[u]:
        if isinstance(data['admin_centers'], str):
            graph.nodes[u]['admin_centers'] = data['admin_centers']
                
    if graph.nodes[v]:
        if isinstance(data['admin_centers'], str):
            graph.nodes[v]['admin_centers'] = data['admin_centers']

In [ ]:
def search_min_distance_18(graph):
    source_nodes = []
    target_nodes = []

    # Формируем список исходных и целевых узлов
    for node, data in graph.nodes(data=True):
        if isinstance(data.get('admin_centers'), str):  # Исходные узлы
            if data['admin_centers'] != 'Saint Petersburg':
                source_nodes.append((node, data['admin_centers']))
            elif data['admin_centers'] == 'Saint Petersburg':  # Целевые узлы
                target_nodes.append((node, data['admin_centers']))

    min_distances = {} # Словарь для хранения минимальных расстояний

    # Общее количество итераций для отображения прогресса
    total_iterations = len(source_nodes) * len(target_nodes)

    progress_bar = tqdm(total=total_iterations)  # Инициализация индикатора прогресса

    # Поиск кратчайшего пути и запоминание минимального расстояния
    for source_node, name in source_nodes:
        for target_node, name_right in target_nodes:
            try: 
                length, path = nx.single_source_dijkstra(graph, source=source_node, target=target_node, weight='length')
                if name not in min_distances or (length/1000) < min_distances[name][0]:
                    min_distances[name] = (length/1000, path)
            except nx.NetworkXNoPath:
                print(f"Путь до {name_right} не найден.")
            progress_bar.update()  # Обновление индикатора прогресса

    progress_bar.close()  # Закрытие индикатора прогресса

    # Преобразование словаря в DataFrame для более удобного отображения
    df = pd.DataFrame(min_distances.values(), index=min_distances.keys())
    df.columns = ['min_distance', 'path']
    return df

result_path_length_18 = search_min_distance_18(graph)

In [ ]:
result_path_length_18

In [ ]:
import folium
mymap = folium.Map()

for index_city in range(len(result_path_length_18['path'])):
    for i in range(len(result_path_length_18['path'][index_city]) - 1):
        edge = (result_path_length_18['path'][index_city][i], result_path_length_18['path'][index_city][i+1])
        if i == 0:
            coordinates = [[graph.nodes[edge[0]]['y'], graph.nodes[edge[0]]['x']], 
                        [graph.nodes[edge[1]]['y'], graph.nodes[edge[1]]['x']]]
            folium.PolyLine(coordinates, color="yellow", weight=2.5, opacity=1).add_to(mymap)
        else:
            coordinates = [[graph.nodes[edge[0]]['y'], graph.nodes[edge[0]]['x']], 
                    [graph.nodes[edge[1]]['y'], graph.nodes[edge[1]]['x']]]
            folium.PolyLine(coordinates, color="red", weight=2.5, opacity=1).add_to(mymap)

mymap

### Тестовая область

In [ ]:
fuel_gdf = gpd.read_file('data/geojsons/fuel.geojson').to_crs(epsg=32636)
stops_gdf = gpd.read_file('data/geojsons/ЖД остановки.geojson').to_crs(epsg=32636)

In [ ]:
gdf_lines_copy = gdf_lines.copy()
gdf_lines_fuel = transferring_points_on_line(fuel_gdf, gdf_lines_copy, column_1='name', column_2='fuel')

In [ ]:
buffer_for_city = {
    'Луга':             20000,
    'Волосово':         20000,
    'Сосновый бор':     20000,
    'Приозерск':        20000,
    'Гатчина':          20000,
    'Кировск':          20000,
    'Всеволожск':       20000,
    'Тосно':            20000,
    'Кириши':           20000,
    'Волхов':           20000,
    'Лодейное Поле':    20000,
    'Тихвин':           20000,
    'Бокситогорск':     40000,
    'Подпорожье':       20000,
    'Кингисепп':        20000,
    'Выборг':           20000,
    'Сланцы':           20000
}



In [ ]:
# Инициализируйте словарь для хранения сопоставления id дороги и города
road_to_city_mapping = {}

for index, city in admin_centers.iterrows():
    # Создаем буфер вокруг каждого города
    buffer_meter = buffer_for_city[city['admin_centers']]
    buffer = city['geometry'].buffer(buffer_meter)
    roads_in_buffer = gdf_lines_fuel.loc[gdf_lines_fuel[gdf_lines_fuel.geometry.intersects(buffer) & gdf_lines_fuel['fuel'].notna()].index]
    
    for road_id in roads_in_buffer.index:
        # Если id дороги уже есть в словаре - добавить город к списку городов для этой id
        if road_id in road_to_city_mapping:
            road_to_city_mapping[road_id].append(city['admin_centers'])
        else:  # Если id дороги еще нет в словаре, создайте новый список городов с этим городом
            road_to_city_mapping[road_id] = [city['admin_centers']]
gdf_lines_fuel['nearest_fuel'] = gdf_lines_fuel.index.map(road_to_city_mapping)

In [ ]:
graph = momepy.gdf_to_nx(gdf_lines_fuel.to_crs(epsg=4326))
for node in graph.nodes:
    graph.nodes()[node]['x'] = node[0]
    graph.nodes()[node]['y'] = node[1]

# Для каждого ребра в графе
for u, v, data in graph.edges(data=True):
    if graph.nodes[u]:
        if isinstance(data['admin_centers'], str):
            graph.nodes[u]['admin_centers'] = data['admin_centers']
        if isinstance(data['nearest_fuel'], list):
            if 'nearest_fuel' not in graph.nodes[u]:
                graph.nodes[u]['nearest_fuel'] = data['nearest_fuel']
                

    if graph.nodes[v]:
        if isinstance(data['admin_centers'], str):
            graph.nodes[v]['admin_centers'] = data['admin_centers']
        if isinstance(data['nearest_fuel'], list):
            if 'nearest_fuel' not in graph.nodes[v]:
                graph.nodes[v]['nearest_fuel'] = data['nearest_fuel']

In [ ]:
def search_min_distance_fuel(graph):
    source_nodes = []
    target_nodes = []

    # Формируем список исходных и целевых узлов
    for node, data in graph.nodes(data=True):
        if isinstance(data.get('admin_centers'), str):  # Исходные узлы
            source_nodes.append((node, data['admin_centers']))
        if isinstance(data.get('nearest_fuel'), list):  # Исходные узлы
            target_nodes.append((node, data['nearest_fuel']))

    min_distances = {} # Словарь для хранения минимальных расстояний

    # Общее количество итераций для отображения прогресса
    total_iterations = len(source_nodes) * len(target_nodes)

    progress_bar = tqdm(total=total_iterations)  # Инициализация индикатора прогресса

    # Поиск кратчайшего пути и запоминание минимального расстояния
    for source_node, name in source_nodes:
        for target_node, name_right in target_nodes:
            if graph.nodes[target_node].get('nearest_fuel') and name in graph.nodes[target_node]['nearest_fuel']:
                try: 
                    length, path = nx.single_source_dijkstra(graph, source=source_node, target=target_node, weight='length')
                    if name not in min_distances or (length/1000) < min_distances[name][0]:
                        min_distances[name] = (length/1000, path)
                except nx.NetworkXNoPath:
                    print(f"Путь до {name_right} не найден.")
            progress_bar.update()  # Обновление индикатора прогресса

    progress_bar.close()  # Закрытие индикатора прогресса

    # Преобразование словаря в DataFrame для более удобного отображения
    df = pd.DataFrame(min_distances.values(), index=min_distances.keys())
    df.columns = ['min_distance', 'path']
    return df

result_path_length_fuel = search_min_distance_fuel(graph)

In [ ]:
result_path_length_fuel['min_distance, min'] = result_path_length_fuel['min_distance'] / 70 * 60
result_path_length_fuel

In [ ]:
gdf_lines_copy = gdf_lines.copy()
gdf_lines_stops = transferring_points_on_line(stops_gdf, gdf_lines_copy, column_1='NAME', column_2='stops_jd')

In [ ]:
# Инициализируйте словарь для хранения сопоставления id дороги и города
road_to_city_mapping = {}

for index, city in admin_centers.iterrows():
    # Создаем буфер вокруг каждого города
    buffer_meter = buffer_for_city[city['admin_centers']]
    buffer = city['geometry'].buffer(buffer_meter)
    roads_in_buffer = gdf_lines_stops.loc[gdf_lines_stops[gdf_lines_stops.geometry.intersects(buffer) & gdf_lines_stops['stops_jd'].notna()].index]
    
    for road_id in roads_in_buffer.index:
        # Если id дороги уже есть в словаре - добавить город к списку городов для этой id
        if road_id in road_to_city_mapping:
            road_to_city_mapping[road_id].append(city['admin_centers'])
        else:  # Если id дороги еще нет в словаре, создайте новый список городов с этим городом
            road_to_city_mapping[road_id] = [city['admin_centers']]
gdf_lines_stops['nearest_stops_jd'] = gdf_lines_stops.index.map(road_to_city_mapping)

In [ ]:
graph = momepy.gdf_to_nx(gdf_lines_stops.to_crs(epsg=4326))
for node in graph.nodes:
    graph.nodes()[node]['x'] = node[0]
    graph.nodes()[node]['y'] = node[1]

# Для каждого ребра в графе
for u, v, data in graph.edges(data=True):
    if graph.nodes[u]:
        if isinstance(data['admin_centers'], str):
            graph.nodes[u]['admin_centers'] = data['admin_centers']
        if isinstance(data['nearest_stops_jd'], list):
            if 'nearest_stops_jd' not in graph.nodes[u]:
                graph.nodes[u]['nearest_stops_jd'] = data['nearest_stops_jd']
                

    if graph.nodes[v]:
        if isinstance(data['admin_centers'], str):
            graph.nodes[v]['admin_centers'] = data['admin_centers']
        if isinstance(data['nearest_stops_jd'], list):
            if 'nearest_stops_jd' not in graph.nodes[v]:
                graph.nodes[v]['nearest_stops_jd'] = data['nearest_stops_jd']

In [ ]:
def search_min_distance_jd(graph):
    source_nodes = []
    target_nodes = []

    # Формируем список исходных и целевых узлов
    for node, data in graph.nodes(data=True):
        if isinstance(data.get('admin_centers'), str):  # Исходные узлы
            source_nodes.append((node, data['admin_centers']))
        if isinstance(data.get('nearest_stops_jd'), list):  # Исходные узлы
            target_nodes.append((node, data['nearest_stops_jd']))

    min_distances = {} # Словарь для хранения минимальных расстояний

    # Общее количество итераций для отображения прогресса
    total_iterations = len(source_nodes) * len(target_nodes)

    progress_bar = tqdm(total=total_iterations)  # Инициализация индикатора прогресса

    # Поиск кратчайшего пути и запоминание минимального расстояния
    for source_node, name in source_nodes:
        for target_node, name_right in target_nodes:
            if graph.nodes[target_node].get('nearest_stops_jd') and name in graph.nodes[target_node]['nearest_stops_jd']:
                try: 
                    length, path = nx.single_source_dijkstra(graph, source=source_node, target=target_node, weight='length')
                    if name not in min_distances or (length/1000) < min_distances[name][0]:
                        min_distances[name] = (length/1000, path)
                except nx.NetworkXNoPath:
                    print(f"Путь до {name_right} не найден.")
            progress_bar.update()  # Обновление индикатора прогресса

    progress_bar.close()  # Закрытие индикатора прогресса

    # Преобразование словаря в DataFrame для более удобного отображения
    df = pd.DataFrame(min_distances.values(), index=min_distances.keys())
    df.columns = ['min_distance', 'path']
    return df

result_path_length_jd = search_min_distance_jd(graph)

In [ ]:
result_path_length_jd['time, min'] = result_path_length_jd[['min_distance']] / 70 * 60
result_path_length_jd

In [ ]:
import folium
mymap = folium.Map()

edges = [(u,v,d) for u, v, d in graph.edges(data=True)  if isinstance(d['nearest_stops_jd'], list)]
# Отображение на карте
for edge in edges:
    coordinates = [[graph.nodes[edge[0]]['y'], graph.nodes[edge[0]]['x']], 
                   [graph.nodes[edge[1]]['y'], graph.nodes[edge[1]]['x']]]
    folium.PolyLine(coordinates, color="blue", weight=2.5, opacity=1, tooltip=f"{graph.nodes[edge[0]]['nearest_stops_jd']}").add_to(mymap)


for index_city in range(len(result_path_length_jd['path'])):
    for i in range(len(result_path_length_jd['path'][index_city]) - 1):
        edge = (result_path_length_jd['path'][index_city][i], result_path_length_jd['path'][index_city][i+1])
        if i == 0:
            coordinates = [[graph.nodes[edge[0]]['y'], graph.nodes[edge[0]]['x']], 
                        [graph.nodes[edge[1]]['y'], graph.nodes[edge[1]]['x']]]
            folium.PolyLine(coordinates, color="yellow", weight=2.5, opacity=1).add_to(mymap)
        else:
            coordinates = [[graph.nodes[edge[0]]['y'], graph.nodes[edge[0]]['x']], 
                    [graph.nodes[edge[1]]['y'], graph.nodes[edge[1]]['x']]]
            folium.PolyLine(coordinates, color="red", weight=2.5, opacity=1).add_to(mymap)

mymap

In [ ]:
gdf_lines_stops[gdf_lines_stops['nearest_stops_jd'].notna()]

In [ ]:
result_path_length
result_path_length_18
result_path_length_fuel

In [ ]:
result_path_length_fuel.reset_index()

In [ ]:
relult_regstatus_spb = result_path_length[['min_distance']].reset_index().merge(result_path_length_18[['min_distance']].reset_index(), left_on='index', right_on='index')

In [ ]:
relult_regstatus_spb_fuel = relult_regstatus_spb.merge(result_path_length_fuel[['min_distance, min']].reset_index(), left_on='index', right_on='index')

In [ ]:
relult_regstatus_spb_fuel_jd = relult_regstatus_spb_fuel.merge(result_path_length_jd[['time, min']].reset_index(), left_on='index', right_on='index')

In [ ]:
relult_regstatus_spb_fuel_jd.rename(columns={
    'min_distance_x': 'REG_STATUS, km',
    'min_distance_y': 'SPb, km',
    'min_distance, min': 'time_GAS, min',
    'time, min': 'time_RR, min'
}, inplace=True)
relult_regstatus_spb_fuel_jd.set_index('index', inplace=True)

In [ ]:
relult_regstatus_spb_fuel_jd.to_csv('data/csv/relult_regstatus_spb_gas_rr.csv')